<a href="https://colab.research.google.com/github/NancWN/ScrumBusters/blob/main/KeywordSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What can it do?:
it takes a predefined list of keywords which is divided between pump and no_pump words. it uses the list to search trough chat data. if a match is found, the group name, the word and label is written down in a seperate list.

Requirements:
1. upload a list with Keywords you want to search for
2. provide chat data from telegram groups in CSV format
3. adjust the path in the code

Lets go! :)

In [116]:
# settings
# feel free to change settings here
# only change other cells if you know what you are doing
keyword_set = 'paper' # choose one of 'paper', 'brainstorm', 'mixed'
message_path = 'data/sample/messages.csv' # put in the path of the message file

In [117]:
# import libraries, load message data and keyword list
import csv
import pandas as pd 

#List of Keywords we want to search
# lists=pd.read_csv("keywordList.csv", sep=';')

lists = pd.read_csv('references/{}_kw.csv'.format(keyword_set),index_col=0)
lists.columns=['pump','no_pump']

pump_words=lists["pump"].dropna()
no_pump_words=lists["no_pump"].dropna()

#telegram chat export (csv-file)
searchlists=pd.read_csv(message_path, sep=',',na_values = ['no info', '.'])
searchlists.text= searchlists.text.astype('str')

In [118]:
# simple function to state if a substring was found in a string
# this is not necessary but it's easier to understand than alternative solutions
def key_found(key,msg):
    if key in str(msg):
        return True
    else:
        return False

In [119]:
# iterate over keyword list
# create new col per keyword to state if the keyword was found in the message
for keyword in pump_words:
    searchlists.loc[:,keyword] = [
        key_found(keyword, msg)
        for msg
        in searchlists.text
    ]
    searchlists.text = [
        msg.replace(keyword,'**'+keyword+'**')
        for msg in searchlists.text
    ]

# same process for no pump words
for keyword in no_pump_words:
    searchlists.loc[:,keyword] = [
        key_found(keyword, msg)
        for msg
        in searchlists.text
    ]
searchlists.text = [
        msg.replace(keyword,'##'+keyword+'##')
        for msg in searchlists.text
    ]

# get the number of keywords found per message
searchlists.loc[:,'n_pump_words'] = (searchlists[pump_words]).sum(axis=1)
searchlists.loc[:,'n_nopump_words'] = (searchlists[no_pump_words]).sum(axis=1)
searchlists.loc[:,'n_words'] = searchlists.n_pump_words + searchlists.n_nopump_words

searchlists.sort_values('n_words',ascending=False,inplace=True)
msg_sus_df = searchlists.loc[searchlists.n_words>0][['text','date','chat','n_words']]

In [122]:
msg_sus_df.to_csv('data/chats/sus_msgs_{}.csv'.format(keyword_set))

TypeError: to_csv() got an unexpected keyword argument 'ignore_index'